# 如何使用LangChain索引API
在这里，我们将介绍使用LangChain索引API的基本工作流程。
索引API允许您从任何来源加载文档并保持与[向量存储](/docs/concepts/vectorstores/)的同步。具体而言，它有助于：
* 避免将重复内容写入向量存储* 避免重写未更改的内容* 避免对未更改内容重复计算嵌入
所有这些都将为您节省时间和金钱，同时提升向量搜索的效果。
至关重要的是，索引API即使对经过多次处理的文档也能正常工作相对于原始源文档的转换步骤（例如通过文本分块）。
## 工作原理
LangChain 索引利用记录管理器（`RecordManager`）来追踪向向量存储中写入的文档。
在索引内容时，系统会为每个文档计算哈希值，并将以下信息存储在记录管理器中：
- 文档哈希值（页面内容和元数据的哈希值）- 写入时间- 源ID —— 每个文档的元数据中应包含相关信息，以便我们能够确定该文档的最终来源
## 删除模式
在将文档索引到向量存储时，可能需要删除向量存储中的某些现有文档。在某些情况下，您可能希望移除与新索引文档来源相同的所有现有文档；而在其他情况下，您可能需要批量删除全部现有文档。索引API的删除模式允许您选择所需的行为：
| 清理模式 | 去重内容 | 可并行处理 | 清理已删除的源文档 | 清理源文档及/或衍生文档的变更 | 清理时机 ||-------------|-----------------------|---------------|----------------------------------|----------------------------------------------------|---------------------|| 无         | ✅                    | ✅            | ❌                               | ❌                                                 | -                  || 增量式 | ✅                    | ✅            | ❌                               | ✅                                                 | 持续式       || 完整        | ✅                    | ❌            | ✅                               | ✅                                                 | 索引结束时         || Scoped_Full | ✅                    | ✅            | ❌                               | ✅                                                 | 索引结束时         |

`None` 不会执行任何自动清理操作，允许用户手动清理旧内容。
`incremental`、`full` 和 `scoped_full` 提供以下自动清理功能：
* 如果源文档或衍生文档的内容发生**变更**，所有3种模式都将清理（删除）先前版本的内容。* 如果源文档已被**删除**（即未包含在正在索引的文档中），`full`清理模式会正确地从向量存储中删除该文档，但`incremental`和`scoped_full`模式则不会执行此操作。
当内容发生变更时（例如源PDF文件被修订），在索引过程中会有一段时期，新旧版本的内容可能同时返回给用户。这种情况发生在新内容写入之后，但旧版本尚未删除之前。
* `增量式`索引通过边写入边持续清理的方式，将这段时间最小化。* `full` 和 `scoped_full` 模式会在所有批次写入完成后执行清理操作。
## 需求
1. 不要将其与已独立于索引API预先填充内容的存储库一起使用，因为记录管理器无法识别先前已插入的记录。2. 仅适用于支持以下功能的LangChain `vectorstore`：* 按ID添加文档（使用带有`ids`参数的`add_documents`方法）* 通过ID删除（使用带有`ids`参数的`delete`方法）
兼容的向量数据库：`Aerospike`、`AnalyticDB`、`AstraDB`、`AwaDB`、`AzureCosmosDBNoSqlVectorSearch`、`AzureCosmosDBVectorSearch`、`AzureSearch`、`Bagel`、`Cassandra`、`Chroma`、`CouchbaseVectorStore`、`DashVector`、`DatabricksVectorSearch`、`DeepLake`、`Dingo`、`ElasticVectorSearch`、`ElasticsearchStore`、`FAISS`、`HanaDB`、`Milvus`、`MongoDBAtlasVectorSearch`、`MyScale`、`OpenSearchVectorSearch`、`PGVector`、`Pinecone`、`Qdrant`、`Redis`、`Rockset`、`ScaNN`、`SingleStoreDB`、`SupabaseVectorStore`、`SurrealDBStore`、`TimescaleVector`、`Vald`、`VDMS`、`Vearch`、`VespaStore`、`Weaviate`、`Yellowbrick`、`ZepVectorStore`、`TencentVectorDB`、`OpenSearchVectorSearch`。  
## 注意
记录管理器依赖基于时间的机制来确定可以清理哪些内容（在使用`full`、`incremental`或`scoped_full`清理模式时）。
如果两个任务连续运行，且第一个任务在时钟时间变更前完成，那么第二个任务可能无法清理内容。
在实际应用中，这不太可能成为问题，原因如下：
1. RecordManager 采用更高精度的时间戳。2. 数据需要在第一次和第二次任务运行之间发生变化，如果任务之间的时间间隔很短，这种情况就不太可能发生。3. 索引任务通常耗时超过几毫秒。

## 快速入门

In [1]:
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

初始化一个向量存储并设置嵌入：

In [2]:
collection_name = "test_index"

embedding = OpenAIEmbeddings()

vectorstore = ElasticsearchStore(
    es_url="http://localhost:9200", index_name="test_index", embedding=embedding
)

使用适当的命名空间初始化记录管理器。
**建议：** 使用一个同时考虑向量存储和向量存储中集合名称的命名空间；例如：'redis/my_docs'、'chromadb/my_docs' 或 'postgres/my_docs'。

In [3]:
namespace = f"elasticsearch/{collection_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)

在使用记录管理器之前创建一个模式。

In [4]:
record_manager.create_schema()

让我们为一些测试文档建立索引：

In [5]:
doc1 = Document(page_content="kitty", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="doggy", metadata={"source": "doggy.txt"})

索引到空向量存储：

In [6]:
def _clear():
    """Hacky helper method to clear content. See the `full` mode section to to understand why it works."""
    index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

### ``None`` 删除模式
此模式不会自动清理旧版本的内容；不过，它仍会处理内容去重。

In [7]:
_clear()

In [8]:
index(
    [doc1, doc1, doc1, doc1, doc1],
    record_manager,
    vectorstore,
    cleanup=None,
    source_id_key="source",
)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [9]:
_clear()

In [10]:
index([doc1, doc2], record_manager, vectorstore, cleanup=None, source_id_key="source")

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

第二次运行时所有内容将被跳过：

In [11]:
index([doc1, doc2], record_manager, vectorstore, cleanup=None, source_id_key="source")

{'num_added': 0, 'num_updated': 0, 'num_skipped': 2, 'num_deleted': 0}

### ``"增量"`` 删除模式

In [12]:
_clear()

In [13]:
index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

重新索引操作将导致两份文档均被**跳过**——同时跳过嵌入操作！

In [14]:
index(
    [doc1, doc2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 0, 'num_updated': 0, 'num_skipped': 2, 'num_deleted': 0}

如果我们不提供任何文档进行增量索引模式，则不会发生任何变化。

In [15]:
index([], record_manager, vectorstore, cleanup="incremental", source_id_key="source")

{'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

如果我们修改了一个文档，新版本将被写入，所有共享同一来源的旧版本将被删除。

In [16]:
changed_doc_2 = Document(page_content="puppy", metadata={"source": "doggy.txt"})

In [17]:
index(
    [changed_doc_2],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 1}

### ``"完全"`` 删除模式
在`full`模式下，用户应将需要建立索引的完整内容全集传入索引函数。
未被传入索引函数且存在于向量存储中的任何文档都将被删除！
此行为有助于处理源文档的删除操作。

In [18]:
_clear()

In [19]:
all_docs = [doc1, doc2]

In [20]:
index(all_docs, record_manager, vectorstore, cleanup="full", source_id_key="source")

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

假设有人删除了第一个文档：

In [21]:
del all_docs[0]

In [22]:
all_docs

[Document(page_content='doggy', metadata={'source': 'doggy.txt'})]

使用完整模式还将清理已删除的内容。

In [23]:
index(all_docs, record_manager, vectorstore, cleanup="full", source_id_key="source")

{'num_added': 0, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 1}

## 源

元数据属性包含一个名为 `source` 的字段。该字段应指向与给定文档关联的*最终*来源。
例如，如果这些文档代表某个父文档的片段，那么这两个文档的 `source` 应该相同，并引用父文档。
通常来说，`source` 字段应当始终被明确指定。仅在你**永不**打算使用 `incremental` 模式，且因某些原因无法正确指定 `source` 字段时，才使用 `None`。

In [24]:
from langchain_text_splitters import CharacterTextSplitter

In [25]:
doc1 = Document(
    page_content="kitty kitty kitty kitty kitty", metadata={"source": "kitty.txt"}
)
doc2 = Document(page_content="doggy doggy the doggy", metadata={"source": "doggy.txt"})

In [26]:
new_docs = CharacterTextSplitter(
    separator="t", keep_separator=True, chunk_size=12, chunk_overlap=2
).split_documents([doc1, doc2])
new_docs

[Document(page_content='kitty kit', metadata={'source': 'kitty.txt'}),
 Document(page_content='tty kitty ki', metadata={'source': 'kitty.txt'}),
 Document(page_content='tty kitty', metadata={'source': 'kitty.txt'}),
 Document(page_content='doggy doggy', metadata={'source': 'doggy.txt'}),
 Document(page_content='the doggy', metadata={'source': 'doggy.txt'})]

In [27]:
_clear()

In [28]:
index(
    new_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 5, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [29]:
changed_doggy_docs = [
    Document(page_content="woof woof", metadata={"source": "doggy.txt"}),
    Document(page_content="woof woof woof", metadata={"source": "doggy.txt"}),
]

这将删除与源文件 `doggy.txt` 关联的旧版本文档，并用新版本替换它们。

In [30]:
index(
    changed_doggy_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 2}

In [31]:
vectorstore.similarity_search("dog", k=30)

[Document(page_content='woof woof', metadata={'source': 'doggy.txt'}),
 Document(page_content='woof woof woof', metadata={'source': 'doggy.txt'}),
 Document(page_content='tty kitty', metadata={'source': 'kitty.txt'}),
 Document(page_content='tty kitty ki', metadata={'source': 'kitty.txt'}),
 Document(page_content='kitty kit', metadata={'source': 'kitty.txt'})]

## 与加载器配合使用
索引可以接受一个可迭代的文档集合，或者任意的加载器。
**注意：** 加载器**必须**正确设置源键。

In [32]:
from langchain_core.document_loaders import BaseLoader


class MyCustomLoader(BaseLoader):
    def lazy_load(self):
        text_splitter = CharacterTextSplitter(
            separator="t", keep_separator=True, chunk_size=12, chunk_overlap=2
        )
        docs = [
            Document(page_content="woof woof", metadata={"source": "doggy.txt"}),
            Document(page_content="woof woof woof", metadata={"source": "doggy.txt"}),
        ]
        yield from text_splitter.split_documents(docs)

    def load(self):
        return list(self.lazy_load())

In [33]:
_clear()

In [34]:
loader = MyCustomLoader()

In [35]:
loader.load()

[Document(page_content='woof woof', metadata={'source': 'doggy.txt'}),
 Document(page_content='woof woof woof', metadata={'source': 'doggy.txt'})]

In [36]:
index(loader, record_manager, vectorstore, cleanup="full", source_id_key="source")

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [37]:
vectorstore.similarity_search("dog", k=30)

[Document(page_content='woof woof', metadata={'source': 'doggy.txt'}),
 Document(page_content='woof woof woof', metadata={'source': 'doggy.txt'})]